In [1]:
!pip install konlpy
!pip install kss
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 725.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kss: filename=kss-4.5.4-py3-none-any.whl size=54465 sha256=eba8b973bc2a1ccd4b66ad50838386144d229ff61afe63117c91c41a3f51c703
  Stored in directory: /root/.cache/pip/wheels/61/7b/ba/e620ef5d96a61cdd83bdee4c2bb4aec8a74de5d72fcbb00e80
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646664 sha256=ab817c311203d9805e308b12553ce7fa3fc88053f9d19afb003180630a7796ef
  Stored in directory: /root/.cache/pip/wheels/5c/6f/b4/ab61b8863d7d8b1409def8ae31adcaa089fa91b8d022ec309d


In [2]:
import numpy as np
import pandas as pd
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from kss import split_sentences
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

from pickle import load
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [6]:
class click_bait:

  ##문장 요약 및 유사도 측정 함수##

  def text2sentences(self, text):
    sentences = split_sentences(text)  #text일 때 문장별로 리스트 만듦
    return sentences

  def build_sent_graph(self, text):
    sentence = self.text2sentences(text)
    tfidf = TfidfVectorizer()
    cnt_vec = CountVectorizer()
    graph_sentence = []
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence

  def get_ranks(self, graph, d=0.85): # d = damping factor
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
    return {idx: r[0] for idx, r in enumerate(ranks)}

  def summarize(self, text, sent_num=4):
    sentences = self.text2sentences(text)
    rank = self.get_ranks(click_bait.build_sent_graph(text))
    sorted_sent_rank_idx = sorted(rank, key=lambda k: rank[k], reverse=True)
    summary = []
    index= []
    for idx in sorted_sent_rank_idx[:sent_num]:
        index.append(idx)
    index.sort()
    for idx in index:
        summary.append(sentences[idx])
    summ = summary[0]+summary[1]+summary[2]+summary[3]
    return summ

  def bart_sum(self, summ):
    tokenizer_b = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
    model_b = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")
    input_ids = tokenizer_b.encode(summ, return_tensors="pt")
    summary_text_ids = model_b.generate(
      input_ids = input_ids,
      bos_token_id = model_b.config.bos_token_id,
      eos_token_id = model_b.config.eos_token_id,
      length_penalty = 0.5,
      max_length = 100,
      min_length = 30,
      num_beams =  4
    )
    res = tokenizer_b.decode(summary_text_ids[0], skip_special_tokens=True)
    return res

  def sim_vs(self, title, content): #자카드 유사도 사용
    answer_string = self.bart_sum(click_bait.summarize(content))
    input_string = title

    intersection_cardinality = len(set.intersection(*[set(answer_string), set(input_string)]))
    union_cardinality = len(set.union(*[set(answer_string), set(input_string)]))
    similar = intersection_cardinality / float(union_cardinality)

    return similar


  ## title_prob 구하기

  def get_t_prob(self, title):
    load_tokenizer = load(open('/content/drive/MyDrive/tobigsNLP1819/tokenizer.pkl', 'rb'))
    x = load_tokenizer.texts_to_sequences([title])
    x = pad_sequences(x, maxlen=20)
    model_t = tf.keras.saving.load_model("/content/drive/MyDrive/tobigsNLP1819/title_prob_model")
    t_prob = model_t.predict(x)[0][0]
    return t_prob


  ## content_prob 구하기

  def get_c_prob(self, title, content):
    p = self.sim_vs(title, content)
    return p


  ## 새로운 기사 제목 생성 함수
  def make_new_title(self, content):
    #content = self.summarize(content)
    tokenizer_b = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
    model_b = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")
    input_ids = tokenizer_b.encode(content, return_tensors="pt")
    summary_text_ids = model_b.generate(
      input_ids = input_ids,
      bos_token_id = model_b.config.bos_token_id,
      eos_token_id = model_b.config.eos_token_id,
      length_penalty = 0.15,
      max_length = 25,
      min_length = 7,
      num_beams =  3
    )
    new_title = tokenizer_b.decode(summary_text_ids[0], skip_special_tokens=True)
    return new_title


  ## 최종 결과

  def isClickbait(self, title, content):
    t_prob = self.get_t_prob(title)
    c_prob = self.get_c_prob(title, content)
    load_scaler = load(open('/content/drive/MyDrive/tobigsNLP1819/standard_scaler.pkl', 'rb'))
    x = [[t_prob, c_prob]]
    x = load_scaler.transform(x)
    clickbait_model = tf.keras.models.load_model("/content/drive/MyDrive/tobigsNLP1819/clickbait_model.h5")
    print("** 낚시성 기사 판별 중 **")
    print("")
    print("제목 :", title)
    print("내용 :", content)
    print("")
    pred = clickbait_model.predict(x)[0]
    print("낚시성 지수 :", 1-pred)

    if pred < 0.5:
      print("낚시성 제목을 가진 기사로 의심됩니다.")
    else:
      print("낚시성 기사일 확률이 낮습니다.")

In [ ]:
## fakenews

# 패키지 로드
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast, BartModel,BartForConditionalGeneration
import math
import re
import logging
from datetime import datetime
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from nltk.translate.bleu_score import corpus_bleu
from transformers import pipeline, AutoTokenizer
import warnings
warnings.filterwarnings(action='ignore')

#model_sts = torch.load("/content/drive/MyDrive/tobigsNLP1819/fake/model.pt")
#data = pd.read_csv('/content/drive/MyDrive/tobigsNLP1819/fake/final_data.csv')

#content = []
#for i in range(len(data)):
#    content.append(str(data['contents'][i]))
#document_embeddings = model_sts.encode(content)

In [ ]:
class fakenews():

    def __init__(self, text):
        self.text = re.sub("(<span class='quot[0-9]'>|\n\r\n|</span>|<br/>|<br />|([^0-9가-힣A-Za-z. ]))","",text)

    def __str__(self):
        return f"해당 클래스는 '{self.text}'라는 문장이 참인지, 거짓인지 판단합니다."

    def summarize(self):
        tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
        model_sm = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
        text = self.text.replace('\n', ' ')
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

        if len(input_ids) > 1024:
            summary_text = '해당 기사 본문 내용이 너무 길어서 요약에 실패했습니다. 좀더 적은 양의 본문 내용을 뽑아주세요'
        else:
            summary_ids = model_sm.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
            summary_text = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

        return summary_text

    def sts(self):

        model_sts = torch.load("/content/drive/MyDrive/tobigsNLP1819/fake/model.pt", map_location=torch.device('cuda'))

        data = pd.read_csv('/content/drive/MyDrive/tobigsNLP1819/fake/final_data.csv')

        content = []
        for i in range(len(data)):
            content.append(str(data['contents'][i]))

        document_embeddings = model_sts.encode(content)

        query_embedding = model_sts.encode(self.summarize())
        cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]
        top_results = torch.topk(cos_scores, k=3)
        num_top_results = sum(top_results[0] > 0.2)

        cos_contents = []
        label_contents = []

        if num_top_results != 0:
            for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
                if score <= 0.6: break
                label = 'True' if data['label'][int(idx)] == 1 else 'False'
                cos_contents.append(content[idx])
                label_contents.append(label)
            return cos_contents, label_contents

        elif num_top_results == 0:
            return "입력 문장과 유사한 문장이 없으므로 확인이 불가능합니다"

    def nli(self, con_text):
        classifier = pipeline("text-classification", model="Huffon/klue-roberta-base-nli", return_all_scores=True)
        tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")
        results = classifier(f'{self.text} {tokenizer.sep_token} {con_text}')[0]
        return results

    def fact(self):
        if self.sts() == "입력 문장과 유사한 문장이 없으므로 확인이 불가능합니다":
            return "입력 문장과 유사한 문장이 없으므로 확인이 불가능합니다"

        else:
            sts_list = []
            for i in self.sts()[0]:
                sts_list.append(i)

            label_list = []
            for i in self.sts()[1]:
                label_list.append(i)

            nli_list = []
            for i in sts_list:
                nli_list.append(self.nli(i))

            fact_list = []

            for i in range(len(nli_list)):
                score = []
                for j in nli_list[i]:
                    score.append(j['score'])

                if score[0]>score[2]:
                    if label_list[i] == 'True':
                        fact_list.append('True')
                    elif label_list[i] == 'False':
                        fact_list.append('False')

                elif score[0]<score[2]:
                    if label_list[i] == 'True':
                        fact_list.append('False')
                    elif label_list[i] == 'False':
                        fact_list.append('True')

            return fact_list

    def isFake(self):
        if self.fact() == "입력 문장과 유사한 문장이 없으므로 확인이 불가능합니다":
            return "입력 문장과 유사한 문장이 없으므로 확인이 불가능합니다"
        else:
            if len(self.fact()) == 1:
                return f"입력 문장'{self.text}'에 대한 진위 여부는 '{self.fact()}'입니다."
            elif len(self.fact()) == 2:
                if self.fact() == ['True','False'] or self.fact() == ['False','True']:
                    return f"입력 문장'{self.text}'에 대한 판단을 유보합니다"
                else:
                    return f"입력 문장'{self.text}'에 대한 진위 여부는 '{self.fact()[0]}'입니다."

            else:
                t = 0
                f = 0
                for i in self.fact():
                    if i == 'True':
                        t += 1
                    else:
                        f += 1
                if t>f:
                    return f"입력 문장'{self.text}'에 대한 진위 여부는 'True'입니다."
                else:
                    return f"입력 문장'{self.text}'에 대한 진위 여부는 'False'입니다."



### 실행 예시

In [7]:
test = pd.read_csv("/content/drive/MyDrive/tobigsNLP1819/last_data.csv")

title = "'갤럭시S21+' 실물 유출…베젤 다이어트 눈길"
content = "삼성전자 차기 플래그십 스마트폰 \'갤럭시S21+\'(플러스) 실물 모습이 등장했다.\n확연하게 얇아진 전면 베젤(테두리)이 시선을 끈다. 29일(현지시각) IT 기기 유출가 마우리QHD 자신의 트위터에 \\'나는 이미지를 유출하지 않는다\\'며 손에 쥔 실물 갤럭시S21+ 모습을 공개했다.\n사진 속 갤럭시S21+는 이번 갤럭시S 시리즈 대표 색상인 보랏빛 팬텀 바이올렛 색상 모델이다.\n제품에서 가장 눈에 띄는 부분은 평평한 화면과 눈에 띄게 얇아진 전면 베젤이다.\n또 카메라 탑재를 위해 화면에 구멍을 내는 펀치 홀도 전작 대비 작아졌다.\n독특한 디자인의 후면 모습은 앞서 등장했던 렌더링(예상도)과 일치한다.\n카메라 모듈은 이른바 카툭튀(카메라가튀어나온모습)가 다소 줄어든 것으로 보인다.\n현재까지 알려진 갤럭시S21+ 주요 사양은 AP(애플리케이션프로세서)로 퀄컴 스냅드래곤 888과 엑시노스 2100이 출시 지역에 따라 달리 탑재되며, 화면은 6.7인치 풀HD+ 해상도 화면이 장착된다.\n메모리(RAM)는 8GB(기가바이트), 배터리는 4800mAh(밀리암페어아워)로 알려졌으며, 카메라는 6400만 화소 메인 카메라를 중심으로 한 트리플(3개) 카메라가 탑재된다.\n갤럭시S21 시리즈는 내년 1월 14일 공개되고 같은 달 29일 출시될 전망이다."

click_bait = click_bait()

click_bait.isClickbait(title, content)

1/1 [==============================] - 1s 668ms/step


For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class



You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


** 낚시성 기사 판별 중 **

제목 : '갤럭시S21+' 실물 유출…베젤 다이어트 눈길
내용 : 삼성전자 차기 플래그십 스마트폰 '갤럭시S21+'(플러스) 실물 모습이 등장했다.
확연하게 얇아진 전면 베젤(테두리)이 시선을 끈다. 29일(현지시각) IT 기기 유출가 마우리QHD 자신의 트위터에 \'나는 이미지를 유출하지 않는다\'며 손에 쥔 실물 갤럭시S21+ 모습을 공개했다.
사진 속 갤럭시S21+는 이번 갤럭시S 시리즈 대표 색상인 보랏빛 팬텀 바이올렛 색상 모델이다.
제품에서 가장 눈에 띄는 부분은 평평한 화면과 눈에 띄게 얇아진 전면 베젤이다.
또 카메라 탑재를 위해 화면에 구멍을 내는 펀치 홀도 전작 대비 작아졌다.
독특한 디자인의 후면 모습은 앞서 등장했던 렌더링(예상도)과 일치한다.
카메라 모듈은 이른바 카툭튀(카메라가튀어나온모습)가 다소 줄어든 것으로 보인다.
현재까지 알려진 갤럭시S21+ 주요 사양은 AP(애플리케이션프로세서)로 퀄컴 스냅드래곤 888과 엑시노스 2100이 출시 지역에 따라 달리 탑재되며, 화면은 6.7인치 풀HD+ 해상도 화면이 장착된다.
메모리(RAM)는 8GB(기가바이트), 배터리는 4800mAh(밀리암페어아워)로 알려졌으며, 카메라는 6400만 화소 메인 카메라를 중심으로 한 트리플(3개) 카메라가 탑재된다.
갤럭시S21 시리즈는 내년 1월 14일 공개되고 같은 달 29일 출시될 전망이다.

1/1 [==============================] - 0s 84ms/step
낚시성 지수 : [0.18090016]
낚시성 기사일 확률이 낮습니다.


In [8]:
click_bait.make_new_title(content)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


"삼성전자의 차기 플래그십 스마트폰 '갤럭시S21+'(플러스)의 확연하게 얇아진 전면"

In [ ]:
#test = pd.read_csv("/content/drive/MyDrive/tobigsNLP1819/last_data.csv")

In [ ]:
#title = "'갤럭시S21+' 실물 유출…베젤 다이어트 눈길"
#content = "삼성전자 차기 플래그십 스마트폰 \'갤럭시S21+\'(플러스) 실물 모습이 등장했다.\n확연하게 얇아진 전면 베젤(테두리)이 시선을 끈다. 29일(현지시각) IT 기기 유출가 마우리QHD 자신의 트위터에 \\'나는 이미지를 유출하지 않는다\\'며 손에 쥔 실물 갤럭시S21+ 모습을 공개했다.\n사진 속 갤럭시S21+는 이번 갤럭시S 시리즈 대표 색상인 보랏빛 팬텀 바이올렛 색상 모델이다.\n제품에서 가장 눈에 띄는 부분은 평평한 화면과 눈에 띄게 얇아진 전면 베젤이다.\n또 카메라 탑재를 위해 화면에 구멍을 내는 펀치 홀도 전작 대비 작아졌다.\n독특한 디자인의 후면 모습은 앞서 등장했던 렌더링(예상도)과 일치한다.\n카메라 모듈은 이른바 카툭튀(카메라가튀어나온모습)가 다소 줄어든 것으로 보인다.\n현재까지 알려진 갤럭시S21+ 주요 사양은 AP(애플리케이션프로세서)로 퀄컴 스냅드래곤 888과 엑시노스 2100이 출시 지역에 따라 달리 탑재되며, 화면은 6.7인치 풀HD+ 해상도 화면이 장착된다.\n메모리(RAM)는 8GB(기가바이트), 배터리는 4800mAh(밀리암페어아워)로 알려졌으며, 카메라는 6400만 화소 메인 카메라를 중심으로 한 트리플(3개) 카메라가 탑재된다.\n갤럭시S21 시리즈는 내년 1월 14일 공개되고 같은 달 29일 출시될 전망이다."

#click_bait.isClickbait(title, content)

1/1 [==============================] - 0s 278ms/step


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


** 낚시성 기사 판별 중 **

제목 : '갤럭시S21+' 실물 유출…베젤 다이어트 눈길
내용 : 삼성전자 차기 플래그십 스마트폰 '갤럭시S21+'(플러스) 실물 모습이 등장했다.
확연하게 얇아진 전면 베젤(테두리)이 시선을 끈다. 29일(현지시각) IT 기기 유출가 마우리QHD 자신의 트위터에 \'나는 이미지를 유출하지 않는다\'며 손에 쥔 실물 갤럭시S21+ 모습을 공개했다.
사진 속 갤럭시S21+는 이번 갤럭시S 시리즈 대표 색상인 보랏빛 팬텀 바이올렛 색상 모델이다.
제품에서 가장 눈에 띄는 부분은 평평한 화면과 눈에 띄게 얇아진 전면 베젤이다.
또 카메라 탑재를 위해 화면에 구멍을 내는 펀치 홀도 전작 대비 작아졌다.
독특한 디자인의 후면 모습은 앞서 등장했던 렌더링(예상도)과 일치한다.
카메라 모듈은 이른바 카툭튀(카메라가튀어나온모습)가 다소 줄어든 것으로 보인다.
현재까지 알려진 갤럭시S21+ 주요 사양은 AP(애플리케이션프로세서)로 퀄컴 스냅드래곤 888과 엑시노스 2100이 출시 지역에 따라 달리 탑재되며, 화면은 6.7인치 풀HD+ 해상도 화면이 장착된다.
메모리(RAM)는 8GB(기가바이트), 배터리는 4800mAh(밀리암페어아워)로 알려졌으며, 카메라는 6400만 화소 메인 카메라를 중심으로 한 트리플(3개) 카메라가 탑재된다.
갤럭시S21 시리즈는 내년 1월 14일 공개되고 같은 달 29일 출시될 전망이다.

1/1 [==============================] - 0s 90ms/step
낚시성 지수 : [0.18090016]
낚시성 기사일 확률이 낮습니다.


In [ ]:
#click_bait.make_new_title(content)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


"삼성전자의 차기 플래그십 스마트폰 '갤럭시S21+'(플러스)의 확연하게 얇아진 전면"

In [ ]:
#title = test['newTitle'][6]
#content = test['newsContent'][6]

#click_bait.isClickbait(title, content)

1/1 [==============================] - 0s 286ms/step


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


** 낚시성 기사 판별 중 **

제목 : 日서 BTS 앨범 발매 당일 45만장 팔려…국외 아티스트 최고 기록
내용 : 10인조 그룹 티알씨엔지(TRCNG) 멤버 두 명이 소속사에 계약 해지를 통보하고 소속사 관계자를 상습아동학대 등 혐의로 고소했다.
이들을 대리하는 법무법인 남강은 18일 \"TRCNG 멤버 조우엽, 양태선은 지난 4일 회사 TS엔터테인먼트에 내용증명을 보내 전속계약 해지를 통보했다\"고 밝혔다.
그러면서 “박모 이사 외 2명에 대해 상습아동학대와 특수폭행치상 등의 혐의로 지난 12일 서울지방경찰청에 고소장을 제출했다”고 덧붙였다.
남강은 보도자료를 통해 “대표이사의 장기 부재 및 소속 연예인들과의 법적 분쟁 등으로 인한 회사의 매니지먼트 능력 상실과 미성년 멤버들에 대한 안무책임자들의 상습아동학대와 특수폭행치상 등의 이유로 계약을 해지했다”고 설명했다.
남강 측은 \"박모 이사가 매 맞기 내기를 해 상습적으로 멤버들을 폭행했을 뿐 아니라 비보이 안무연습 중에 부상을 당해도 회사에서 아무런 조치를 취해주지 않았으며 조우엽은 지난 6월 윤모 안무팀장에게 철제 의자로 맞아 전치 2주 상처를 입었다\"고 주장했다.
또 \"(미성년인) 멤버들을 매일 오후 5시부터 다음 날 오전 5시까지 잠도 재우지 않고 안무 연습을 시키고 다시 아침 10시까지 출근하게 해 개인연습을 시키다가 오후 5시가 되면 안무연습을 시키는 등 가혹한 연습을 시켜 두 멤버는 학교를 제대로 다닐 수 없었다”고 밝혔다.
이에 대해 소속사 TS엔터테인먼트는입장문을 내고“이들이 주장하는 내용은 태선, 우엽의 일방적인 주장으로써 TRCNG 멤버 모두가 동의하고 있는 내용이 아니다”라고 반박했다.
소속사는“태선, 우엽의 행동은 TRCNG의 활동 전체에 피해를 주고 있으며 당사의 명예를 훼손시킨 부분 등 손실을 일으키고 있는 부분에 대해서 법적 조치를 취할 예정”이라고 말했다.
TRCNG는 2017년 미니앨범 '뉴 제너래이션'(NEW GENERATION)으로 데뷔했다.
멤버들은 모두 2000

In [ ]:
#click_bait.make_new_title(content)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


'10인조 그룹 티알씨엔지(TRCNG) 멤버 두 명이 소속사에 계약 해지를'